In [1]:
import QuantLib as ql
import matplotlib.pyplot as plt
import numpy as np

In [2]:
spread = 0.0458
recovery = 0.25
trade_date = ql.Date(9, 5, 2024)
termDate = ql.Date(20, 6, 2029)

ql.Settings.instance().evaluationDate = trade_date

ql.IborCoupon.createAtParCoupons()

dep_tenors = [1, 2, 3, 6, 12]
dep_quotes = [
    0.053209,
    0.053240,
    0.053256,
    0.05293,
    0.051551,
]
isdaRateHelpers = [
    ql.DepositRateHelper(
        dep_quotes[i],
        dep_tenors[i] * ql.Period(ql.Monthly),
        2,
        ql.WeekendsOnly(),
        ql.ModifiedFollowing,
        False,
        ql.Actual360(),
    )
    for i in range(len(dep_tenors))
]

swap_tenors = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30]
swap_quotes = [
    0.047535,
    0.045104,
    0.043555,
    0.042585,
    0.041992,
    0.041649,
    0.041395,
    0.041235,
    0.041149,
    0.041115,
    0.041111,
    0.040735,
    0.039775,
    0.038805,
]

isda_ibor = ql.IborIndex(
    "IsdaIbor",
    3 * ql.Period(ql.Monthly),
    2,
    ql.USDCurrency(),
    ql.WeekendsOnly(),
    ql.ModifiedFollowing,
    False,
    ql.Actual360(),
)
isdaRateHelpers = isdaRateHelpers + [
    ql.SwapRateHelper(
        swap_quotes[i],
        swap_tenors[i] * ql.Period(ql.Annual),
        ql.WeekendsOnly(),
        ql.Semiannual,
        ql.ModifiedFollowing,
        ql.Thirty360(ql.Thirty360.BondBasis),
        isda_ibor,
    )
    for i in range(len(swap_tenors))
]

spot_date = ql.WeekendsOnly().advance(trade_date, 2 * ql.Period(ql.Daily))

swap_curve = ql.PiecewiseFlatForward(trade_date, isdaRateHelpers, ql.Actual365Fixed())
discountCurve = ql.YieldTermStructureHandle(swap_curve)
probabilityCurve = ql.RelinkableDefaultProbabilityTermStructureHandle()


upfront_date = ql.WeekendsOnly().advance(trade_date, 3 * ql.Period(ql.Daily))
cdsSchedule = ql.Schedule(
    trade_date,
    termDate,
    3 * ql.Period(ql.Monthly),
    ql.WeekendsOnly(),
    ql.Following,
    ql.Unadjusted,
    ql.DateGeneration.CDS,
    False,
)

quotedTrade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,
    10000000,
    0,
    spread,
    cdsSchedule,
    ql.Following,
    ql.Actual360(),
    True,
    True,
    trade_date,
    upfront_date,
    ql.FaceValueClaim(),
    ql.Actual360(True),
)

h = quotedTrade.impliedHazardRate(
    0,
    discountCurve,
    ql.Actual365Fixed(),
    recovery,
    1e-10,
    ql.CreditDefaultSwap.ISDA,
)

probabilityCurve.linkTo(
    ql.FlatHazardRate(
        0,
        ql.WeekendsOnly(),
        ql.QuoteHandle(ql.SimpleQuote(h)),  # ql.makeQuoteHandle(h),
        ql.Actual365Fixed(),
    )
)

engine = ql.IsdaCdsEngine(probabilityCurve, recovery, discountCurve)
conventionalTrade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,
    10_000_000,
    0,
    0.01,
    cdsSchedule,
    ql.Following,
    ql.Actual360(),
    True,
    True,
    trade_date,
    upfront_date,
    ql.FaceValueClaim(),
    ql.Actual360(True),
)
conventionalTrade.setPricingEngine(engine)

upfront = conventionalTrade.notional() * conventionalTrade.fairUpfront()

In [3]:
upfront

1426417.9885900372